# Import Neccessry libraries

In [56]:
from nltk.corpus import stopwords,reuters
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
stop_words = stopwords.words("english")
from sklearn.metrics import accuracy_score , classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import re
import string

# Represent Train And Test Docs


In [30]:
#extracting train and test documents
#train_documents
train_docs_id = list(filter(lambda doc: doc.startswith("train"),reuters.fileids()));
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
print('Number of docs in the train set: ' + str(len(train_docs)))
#test_documents
test_docs_id = list(filter(lambda doc: doc.startswith("test"),reuters.fileids()));
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]
print('Number of docs in the test set: ' + str(len(test_docs)))


Number of docs in the train set: 7769
Number of docs in the test set: 3019


# Cleaning Documents


In [32]:
codes = ['\r', '\n', '\t','lt']
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower() #Make text lowercase
    text = re.sub('\[.*?\]', '',str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text)) #remove punctuation
    text = re.sub('\w*\d\w*', '', str(text)) #remove words containing numbers
    text = re.sub('[‘’“”…]', '', str(text))
    text = re.sub(r'dlrs', 'dollar', text)  # replace dlrs abreviation 
    text = re.sub(r'pct', 'percent', text)  # replace pct abreviation  
    for code in codes:
        text = re.sub(code, ' ', text)  # get rid of escape codes
    text = re.sub('\s+', ' ', text) # replace multiple spacess with one space   
    return text

In [33]:
#cleaning training and testing documents
cleaned_train_documents = []
for i in range(0,len(train_docs)):    
    cleaned_train_documents.append(clean_text(str(train_docs[i])))
cleaned_test_documents = []
for i in range(0,len(test_docs)):    
    cleaned_test_documents.append(clean_text(str(test_docs[i])))

# Tokenization


In [35]:
def toknize(text):
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text) #split document into individual words(tokens)
    tokens= [word for word in tokens if word not in stop_words] #filter out stop words if requested
    tokens = [word for word in tokens if word.isalpha()] #remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if len(word) > 2] #filter out tokens that are one or two characters long
    tokens = [word for word in tokens if len(word) < 21] # filter out tokens that are more than twenty characters long
    # recreate the document string from parsed words
    text = ''
    for token in tokens:
        text = text + ' ' + token
    return tokens,text

In [48]:
train_text = [] # list of document strings for sklearn TF-IDF
train_tokens = []  # list of token lists
for doc in cleaned_train_documents:
    text_string = doc
    # parse words one at a time in document string
    tokens,text_string = toknize(text_string)
    train_tokens.append(tokens)
    train_text.append(text_string)


In [49]:
test_tokens = []  # list of token lists
test_text = [] # list of document strings for sklearn TF-IDF
for doc in cleaned_test_documents:
    text_string = doc
    # parse words one at a time in document string
    tokens,text_string = toknize(text_string)
    test_tokens.append(text_string)
    test_text.append(text_string)

# Applying  (Count Vectorizer)(tf)


In [50]:
count_vectorizer = CountVectorizer(tokenizer=word_tokenize,max_df=0.95, min_df=3,max_features=1000)
# Learn and transform train documents
vectorised_train_documents = count_vectorizer.fit_transform(train_text)
# transform test documents
vectorised_test_documents= count_vectorizer.transform(test_text)

# Transform multilabel labels


In [51]:
#encode multi-label per instance 
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(doc_id)
for doc_id in train_docs_id])
test_labels = mlb.transform([reuters.categories(doc_id)
for doc_id in test_docs_id])

# Train and Evaluate Classifiers


In [52]:
ModelsPerformance = {}

def metricsReport(modelName, test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    hamLoss = hamming_loss(test_labels, predictions)
    print("------" + modelName + " Model Metrics-----")
    print("Accuracy: {:.4f}\nHamming Loss: {:.4f}\nPrecision:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nRecall:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nF1-measure:\n  - Macro: {:.4f}\n  - Micro: {:.4f}"\
          .format(accuracy, hamLoss, macro_precision, micro_precision, macro_recall, micro_recall, macro_f1, micro_f1))
    ModelsPerformance[modelName] = micro_f1

# Random Forest Classifier

In [57]:
rfClassifier = RandomForestClassifier(n_jobs=-1)
rfClassifier.fit(vectorised_train_documents, train_labels)
rfPreds = rfClassifier.predict(vectorised_test_documents)
metricsReport("Random Forest", test_labels, rfPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Random Forest Model Metrics-----
Accuracy: 0.7122
Hamming Loss: 0.0050
Precision:
  - Macro: 0.4245
  - Micro: 0.9562
Recall:
  - Macro: 0.1496
  - Micro: 0.6651
F1-measure:
  - Macro: 0.1994
  - Micro: 0.7845


# Decision Tree Classifier

In [63]:
dtClassifier = DecisionTreeClassifier()
dtClassifier.fit(vectorised_train_documents, train_labels)
dtPreds = dtClassifier.predict(vectorised_test_documents)
metricsReport("Decision Tree", test_labels, dtPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Decision Tree Model Metrics-----
Accuracy: 0.7304
Hamming Loss: 0.0065
Precision:
  - Macro: 0.3997
  - Micro: 0.7778
Recall:
  - Macro: 0.3194
  - Micro: 0.7404
F1-measure:
  - Macro: 0.3396
  - Micro: 0.7586


# Support Vector Machine Classifier 

In [61]:
from sklearn.preprocessing import MaxAbsScaler  #if its a dense matrix else use MaxAbsScaler in case of sparse matrix
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(vectorised_train_documents)
X_test = scaler.transform(vectorised_test_documents)
svmClassifier = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svmClassifier.fit(X_train, train_labels)

svmPreds = svmClassifier.predict(X_test)
metricsReport("SVC Sq. Hinge Loss", test_labels, svmPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------SVC Sq. Hinge Loss Model Metrics-----
Accuracy: 0.7406
Hamming Loss: 0.0046
Precision:
  - Macro: 0.5044
  - Micro: 0.9000
Recall:
  - Macro: 0.2924
  - Micro: 0.7473
F1-measure:
  - Macro: 0.3487
  - Micro: 0.8166


In [62]:
powerSetSVC = LabelPowerset(LinearSVC())
powerSetSVC.fit(vectorised_train_documents, train_labels)

powerSetSVCPreds = powerSetSVC.predict(vectorised_test_documents)
metricsReport("Power Set SVC", test_labels, powerSetSVCPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


------Power Set SVC Model Metrics-----
Accuracy: 0.7920
Hamming Loss: 0.0055
Precision:
  - Macro: 0.4618
  - Micro: 0.8118
Recall:
  - Macro: 0.3570
  - Micro: 0.7786
F1-measure:
  - Macro: 0.3818
  - Micro: 0.7948


C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Base Classifier

In [64]:
nbClassifier = OneVsRestClassifier(MultinomialNB())
nbClassifier.fit(vectorised_train_documents, train_labels)

nbPreds = nbClassifier.predict(vectorised_test_documents)
metricsReport("Multinomial NB", test_labels, nbPreds)

------Multinomial NB Model Metrics-----
Accuracy: 0.5449
Hamming Loss: 0.0344
Precision:
  - Macro: 0.1230
  - Micro: 0.2788
Recall:
  - Macro: 0.6886
  - Micro: 0.9423
F1-measure:
  - Macro: 0.1885
  - Micro: 0.4302


In [65]:
print("  Model Name " + " "*10 + "| Micro-F1 Score")
print("-------------------------------------------")
for key, value in ModelsPerformance.items():
    print("  " + key, " "*(20-len(key)) + "|", value)
    print("-------------------------------------------")

  Model Name           | Micro-F1 Score
-------------------------------------------
  Random Forest        | 0.7844990548204159
-------------------------------------------
  SVC Sq. Hinge Loss   | 0.816576681745221
-------------------------------------------
  Power Set SVC        | 0.7948193592365371
-------------------------------------------
  Decision Tree        | 0.7586206896551725
-------------------------------------------
  Multinomial NB       | 0.43024390243902433
-------------------------------------------
